In [ ]:
import ee

ee.Initialize()
from IPython.display import display, Image

In [ ]:
scale = 20

In [ ]:
roi = ee.FeatureCollection("users/nkeikon/myanmar_sr/area2")
s2TNI = ee.Image("users/nkeikon/myanmar_sr/s2image").clip(roi)
classified = ee.Image("users/nkeikon/myanmar_sr/area2_classified")
PA = ee.FeatureCollection("WCMC/WDPA/current/polygons")
palm = ee.FeatureCollection("users/nkeikon/myanmar_sr/palm_area2")
rubber = ee.FeatureCollection("users/nkeikon/myanmar_sr/rubber_area2")
other = ee.FeatureCollection("users/nkeikon/myanmar_sr/other_area2")
bare = ee.FeatureCollection("users/nkeikon/myanmar_sr/bare_area2")
water = ee.FeatureCollection("users/nkeikon/myanmar_sr/water_area2")
shrub = ee.FeatureCollection("users/nkeikon/myanmar_sr/shrub_area2")

In [ ]:
Image(url=classified.randomVisualizer().getThumbUrl({"size": "400"}))

In [ ]:
# Calculate the area by class
names = ["1 oil palm", "2 rubber", "3 other", "4 shrub", "5 bare", "6 water"]
count = classified.eq([1, 2, 3, 4, 5, 6]).rename(names)
total = count.multiply(ee.Image.pixelArea())
areaM2 = total.reduceRegion(ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3)
areaPixel = count.reduceRegion(ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3)

# Create feature collections
featureM2 = ee.Feature(None)
featureM2 = featureM2.set("areaM2", areaM2)
area_M2 = ee.FeatureCollection(featureM2)

featurePixel = ee.Feature(None)
featurePixel = featurePixel.set("areaPixel", areaPixel)
area_Pixel = ee.FeatureCollection(featurePixel)

# Export as csv
export1 = ee.batch.Export.table.toDrive(
    collection=area_M2,
    description="export_area2_m2",
    fileNamePrefix="Kawthaung_area2_m2",
)
export1.start()
export2 = ee.batch.Export.table.toDrive(
    collection=area_Pixel,
    description="export_area2_pixel",
    fileNamePrefix="Kawthaung_area2_pixel",
)
export2.start()

In [ ]:
# Pixel counts in all reference data - to correct bias later
names = ["1 oil palm", "2 rubber", "3 other", "4 shrub", "5 bare", "6 water"]
palm_count = classified.clip(palm).eq([1, 2, 3, 4, 5, 6]).rename(names)
rubber_count = classified.clip(rubber).eq([1, 2, 3, 4, 5, 6]).rename(names)
other_count = classified.clip(other).eq([1, 2, 3, 4, 5, 6]).rename(names)
shrub_count = classified.clip(shrub).eq([1, 2, 3, 4, 5, 6]).rename(names)
bare_count = classified.clip(bare).eq([1, 2, 3, 4, 5, 6]).rename(names)
water_count = classified.clip(water).eq([1, 2, 3, 4, 5, 6]).rename(names)

palm_area = palm_count.reduceRegion(
    ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3
)
rubber_area = rubber_count.reduceRegion(
    ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3
)
other_area = other_count.reduceRegion(
    ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3
)
shrub_area = shrub_count.reduceRegion(
    ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3
)
bare_area = bare_count.reduceRegion(
    ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3
)
water_area = water_count.reduceRegion(
    ee.Reducer.sum(), roi, scale, None, None, False, 1e13, 3
)

feature = ee.Feature(None)
feature = feature.set("palm", palm_area)
feature = feature.set("rubber", rubber_area)
feature = feature.set("other", other_area)
feature = feature.set("shrub", shrub_area)
feature = feature.set("bare", bare_area)
feature = feature.set("water", water_area)
area_pixel_results = ee.FeatureCollection(feature)

export3 = ee.batch.Export.table.toDrive(
    collection=area_pixel_results,
    description="export_area2_reference_pixel",
    fileNamePrefix="area2_reference_pixel",
)
export3.start()

In [ ]:
# query current tasks
tasks = ee.batch.Task.list()
print(tasks[0])
print(tasks[1])
print(tasks[2])